In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")

print("Path to dataset files:", path)

100%|██████████| 25.7M/25.7M [00:00<00:00, 157MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews/versions/1


In [ ]:
import pandas as pd

# Đường dẫn mới của dataset
dataset_path = '/root/.cache/kagglehub/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews/versions/1/IMDB Dataset.csv'

# Đọc dataset
df = pd.read_csv(dataset_path)

# Hiển thị vài dòng đầu để kiểm tra
print(df.head())

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [ ]:
# Đọc dữ liệu từ file CSV (giả sử bạn đã tải về từ Kaggle)
# Giả sử dataframe tên là `df` và có cột `review` và `sentiment`

# Loại bỏ các ký tự không cần thiết, viết thường và loại bỏ stop words
def simple_preprocess(text):
    text = re.sub(r'[^\w\s]', '', text)  # Loại bỏ dấu câu
    text = text.lower()  # Chuyển thành chữ thường
    return text

# Áp dụng hàm xử lý cho toàn bộ dữ liệu
df['processed_review'] = df['review'].apply(simple_preprocess)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Sử dụng CountVectorizer (hoặc bạn có thể thay thế bằng TfidfVectorizer)
vectorizer = CountVectorizer(max_features=5000)  # Giới hạn 5000 từ phổ biến nhất

# Chuyển đổi văn bản thành dạng vector
X = vectorizer.fit_transform(df['processed_review']).toarray()
y = df['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)  # Chuyển nhãn thành 0 và 1

# Chia tập dữ liệu thành train và test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Tải stop words và khởi tạo stemmer
stop_words = set(stopwords.words("english"))
stemmer = PorterStemmer()

def clean_text(text):
    # Chuyển thành chữ thường
    text = text.lower()
    # Loại bỏ thẻ HTML
    text = re.sub(r'<.*?>', ' ', text)
    # Loại bỏ ký tự đặc biệt và dấu câu
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Loại bỏ khoảng trắng thừa
    text = re.sub(r'\s+', ' ', text).strip()
    # Loại bỏ stop words và áp dụng stemming
    words = text.split()
    words = [stemmer.stem(word) for word in words if word not in stop_words]
    text = ' '.join(words)
    return text

# Áp dụng hàm làm sạch cho cột review
df['cleaned_review'] = df['review'].apply(clean_text)

# Kiểm tra kết quả
print(df[['review', 'cleaned_review']].head())

                                              review  \
0  One of the other reviewers has mentioned that ...   
1  A wonderful little production. <br /><br />The...   
2  I thought this was a wonderful way to spend ti...   
3  Basically there's a family where a little boy ...   
4  Petter Mattei's "Love in the Time of Money" is...   

                                      cleaned_review  
0  one review mention watch oz episod youll hook ...  
1  wonder littl product film techniqu unassum old...  
2  thought wonder way spend time hot summer weeke...  
3  basic there famili littl boy jake think there ...  
4  petter mattei love time money visual stun film...  


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Chia dữ liệu thành train và test
X_train, X_test, y_train, y_test = train_test_split(df['cleaned_review'], df['sentiment'], test_size=0.2, random_state=42)

# Khởi tạo TfidfVectorizer
tfidf = TfidfVectorizer(max_features=5000)  # Giới hạn số lượng từ tối đa để giảm độ phức tạp

# Vector hóa dữ liệu
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression

# Khởi tạo mô hình Logistic Regression
model = LogisticRegression(max_iter=100)

# Huấn luyện mô hình
model.fit(X_train_tfidf, y_train)

LogisticRegression()

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Dự đoán trên tập kiểm tra
y_pred = model.predict(X_test_tfidf)

# Tính toán độ chính xác
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# In báo cáo chi tiết
print(classification_report(y_test, y_pred, target_names=["Negative", "Positive"]))

Accuracy: 0.8858
              precision    recall  f1-score   support

    Negative       0.90      0.87      0.88      4961
    Positive       0.87      0.90      0.89      5039

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000



In [ ]:
# Câu nhận xét tích cực để kiểm tra mô hình
new_reviews = ["I absolutely loved this movie!", "This was a waste of time."]

# Làm sạch và vector hóa các câu nhận xét mới
new_reviews_cleaned = [clean_text(review) for review in new_reviews]
new_reviews_tfidf = tfidf.transform(new_reviews_cleaned)

# Dự đoán xác suất
probs = model.predict_proba(new_reviews_tfidf)  # Sử dụng predict_proba nếu mô hình hỗ trợ

# In ra xác suất
for review, prob in zip(new_reviews, probs):
    print(f"Review: '{review}' --> Probability (Positive): {prob[1]:.4f}, Probability (Negative): {prob[0]:.4f}")

Review: 'I absolutely loved this movie!' --> Probability (Positive): 0.8750, Probability (Negative): 0.1250
Review: 'This was a waste of time.' --> Probability (Positive): 0.0009, Probability (Negative): 0.9991


In [ ]:
for review, prob in zip(new_reviews, probs):
    sentiment = "Positive" if prob[1] > prob[0] else "Negative"
    print(f"Review: '{review}' --> Sentiment: {sentiment}")

Review: 'I absolutely loved this movie!' --> Sentiment: Positive
Review: 'This was a waste of time.' --> Sentiment: Negative


In [ ]:
# Một số câu nhận xét để kiểm tra vectorizer
test_reviews = [
    "The movie was fantastic and full of great performances!",
    "I didn't enjoy the movie; it was boring.",
    "An average film with decent visuals but weak plot."
]

# Làm sạch các câu nhận xét
test_reviews_cleaned = [clean_text(review) for review in test_reviews]

# Chuyển đổi các câu nhận xét đã làm sạch bằng tfidf
test_reviews_tfidf = tfidf.transform(test_reviews_cleaned)

# Kiểm tra xem TfidfVectorizer có hoạt động đúng không
print("Shape of transformed data:", test_reviews_tfidf.shape)
print("Transformed data (first row):", test_reviews_tfidf[0].toarray())


Shape of transformed data: (3, 5000)
Transformed data (first row): [[0. 0. 0. ... 0. 0. 0.]]


In [ ]:
import joblib

joblib.dump(tfidf, 'tfidf_vectorizer.pkl')


['tfidf_vectorizer.pkl']

In [ ]:
# Tải lại TfidfVectorizer từ file
tfidf_loaded = joblib.load('tfidf_vectorizer.pkl')

# Kiểm tra xem tfidf_loaded có hoạt động giống như tfidf ban đầu không
test_reviews_tfidf_loaded = tfidf_loaded.transform(test_reviews_cleaned)
print("Shape of transformed data (using loaded tfidf):", test_reviews_tfidf_loaded.shape)
print("Transformed data (first row using loaded tfidf):", test_reviews_tfidf_loaded[0].toarray())


Shape of transformed data (using loaded tfidf): (3, 5000)
Transformed data (first row using loaded tfidf): [[0. 0. 0. ... 0. 0. 0.]]


In [ ]:
joblib.dump(model, 'sentiment_model.pkl')


['sentiment_model.pkl']

In [ ]:
print(df.columns)

Index(['review', 'sentiment', 'processed_review', 'cleaned_review'], dtype='object')
